In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/archive (3).zip" -d /content/dataset1/

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import torch
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import os
import segmentation_models_pytorch as smp

class SegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.image_files = sorted(os.listdir(images_dir))
        self.mask_files = sorted(os.listdir(masks_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Загружаем изображения и маски
        image_path = os.path.join(self.images_dir, self.image_files[idx])
        mask_path = os.path.join(self.masks_dir, self.mask_files[idx])

        image = Image.open(image_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')

        # Применение преобразований
        if self.transform:
            image = self.transform(image)

        # Преобразуем маску в тензор и убираем лишнюю размерность
        mask = transforms.ToTensor()(mask)  
        mask = mask.unsqueeze(0)

        return image, mask

In [ ]:
image_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

])

In [ ]:
# Пути к папкам с изображениями и масками
images_dir = '/content/dataset1/Forest Segmented/Forest Segmented/images'
masks_dir = '/content/dataset1/Forest Segmented/Forest Segmented/masks'

# Получаем список файлов
image_files = sorted(os.listdir(images_dir))
mask_files = sorted(os.listdir(masks_dir))


train_images, valid_images, train_masks, valid_masks = train_test_split(
    image_files, mask_files, test_size=0.2, random_state=42
)

# Создание датасетов
train_dataset = SegmentationDataset(
    images_dir=images_dir,
    masks_dir=masks_dir,
    transform=image_transform
)

valid_dataset = SegmentationDataset(
    images_dir=images_dir,
    masks_dir=masks_dir,
    transform=image_transform
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Valid dataset size: {len(valid_dataset)}")

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)

In [ ]:
!pip install segmentation-models-pytorch torch torchvision scikit-learn matplotlib

In [ ]:
# Загрузка предобученной модели U-Net
model = smp.Unet(
    encoder_name='resnet50',        # Энкодер
    encoder_weights='imagenet',    # Предобученные веса на ImageNet
    in_channels=3,                 # Количество входных каналов (RGB = 3)
    classes=1                      # Количество классов (для бинарной сегментации = 1)
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True )


In [ ]:
import numpy as np

# Функция для вычисления IoU
def calculate_iou(true_mask, pred_mask):
    intersection = np.sum(true_mask * pred_mask)
    union = np.sum(true_mask) + np.sum(pred_mask) - intersection
    return intersection / union if union != 0 else 0

# Функция для вычисления Dice Coefficient
def calculate_dice(true_mask, pred_mask):
    intersection = np.sum(true_mask * pred_mask)
    return (2.0 * intersection) / (np.sum(true_mask) + np.sum(pred_mask) + 1e-7)

# Функция для вычисления Accuracy
def calculate_accuracy(true_mask, pred_mask):
    correct_pixels = np.sum(true_mask == pred_mask)
    total_pixels = true_mask.size
    return correct_pixels / total_pixels

In [ ]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_iou = 0.0
        train_dice = 0.0
        train_accuracy = 0.0

        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)

            optimizer.zero_grad()
            outputs = model(images)

            outputs = outputs.unsqueeze(1)

            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            preds = torch.sigmoid(outputs).detach().cpu().numpy() > 0.5
            true_masks = masks.detach().cpu().numpy()

            train_iou += np.mean([calculate_iou(true_mask, pred_mask)
                                  for true_mask, pred_mask in zip(true_masks, preds)])
            train_dice += np.mean([calculate_dice(true_mask, pred_mask)
                                   for true_mask, pred_mask in zip(true_masks, preds)])
            train_accuracy += np.mean([calculate_accuracy(true_mask, pred_mask)
                                       for true_mask, pred_mask in zip(true_masks, preds)])

        # Валидация
        model.eval()
        valid_loss = 0.0
        valid_iou = 0.0
        valid_dice = 0.0
        valid_accuracy = 0.0
        with torch.no_grad():
            for images, masks in valid_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)

                outputs = outputs.unsqueeze(1)  # Добавляем размерность канала
                loss = criterion(outputs, masks)
                valid_loss += loss.item()

                preds = torch.sigmoid(outputs).detach().cpu().numpy() > 0.5
                true_masks = masks.detach().cpu().numpy()

                valid_iou += np.mean([calculate_iou(true_mask, pred_mask)
                                      for true_mask, pred_mask in zip(true_masks, preds)])
                valid_dice += np.mean([calculate_dice(true_mask, pred_mask)
                                       for true_mask, pred_mask in zip(true_masks, preds)])
                valid_accuracy += np.mean([calculate_accuracy(true_mask, pred_mask)
                                           for true_mask, pred_mask in zip(true_masks, preds)])

        train_loss /= len(train_loader)
        train_iou /= len(train_loader)
        train_dice /= len(train_loader)
        train_accuracy /= len(train_loader)

        valid_loss /= len(valid_loader)
        valid_iou /= len(valid_loader)
        valid_dice /= len(valid_loader)
        valid_accuracy /= len(valid_loader)

        print(f"Epoch {epoch + 1}/{num_epochs}, "
              f"Train Loss: {train_loss:.4f}, Train IoU: {train_iou:.4f}, Train Dice: {train_dice:.4f}, Train Accuracy: {train_accuracy:.4f}, "
              f"Valid Loss: {valid_loss:.4f}, Valid IoU: {valid_iou:.4f}, Valid Dice: {valid_dice:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

In [ ]:
train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=10)

In [ ]:
image_path = '/content/dataset1/Forest Segmented/Forest Segmented/images/114433_sat_60.jpg'
mask_path = '/content/dataset1/Forest Segmented/Forest Segmented/masks/114433_mask_60.jpg'

# Загрузка и предобработка изображения
image = Image.open(image_path).convert("RGB")
image_tensor = transform(image).unsqueeze(0).to(device)

# Загрузка истинной маски
true_mask = Image.open(mask_path).convert("L")  # Конвертация в оттенки серого
true_mask = np.array(true_mask.resize((256, 256))) / 255  # Изменение размера и нормализация [0, 1]

# Инференс
model.eval()
with torch.no_grad():
    output = model(image_tensor)
    pred_mask = torch.sigmoid(output).cpu().numpy()[0][0] > 0.5  # Бинаризация предсказания

plt.figure(figsize=(18, 6))

# Исходное изображение
plt.subplot(1, 3, 1)
plt.imshow(image)
plt.title("Input Image")
plt.axis("off")

# Истинная маска
plt.subplot(1, 3, 2)
plt.imshow(true_mask, cmap="gray")
plt.title("Ground Truth Mask")
plt.axis("off")

# Предсказанная маска
plt.subplot(1, 3, 3)
plt.imshow(pred_mask, cmap="gray")
plt.title("Predicted Mask")
plt.axis("off")

plt.show()